# Mise à jour de la BDD de NFT
Source: [binance.com](https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1)

In [1]:
import os
import time
import numpy as np
from utils import *
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

## 1. Ouverture du marketplace

- Paramétrage du navigateur distant
- Chargement de la page d'accueil
- Activation des cookies 

In [2]:
%%time
start_url = 'https://www.binance.com/en/nft/market?currency=&mediaType=&tradeType=&amountFrom=&amountTo=&categorys=&keyword=&page=1&rows=16&productIds=&order=list_time%40-1'
browser = webdriver.Remote("http://selenium:4444/wd/hub", DesiredCapabilities.FIREFOX)
browser.get(start_url)
cookies = browser.find_element_by_xpath("//button[contains(text(),'Accept')]")
cookies.click()

CPU times: user 36.4 ms, sys: 38.4 ms, total: 74.8 ms
Wall time: 1min 33s


## 2. Défilement des pages sur le marketplace
 - Choix du nombre de nft à récupérer
 - Chargement des pages contenant les nfts à récuperer 

In [4]:
%%time

MAX_NFT_NB = 25#select_number()
CURRENT_PAGE_NUMBER = int(browser.current_url.split("page=")[-1].split("&")[0])
NFT_NUMBER_PER_PAGE = int(browser.current_url.split("rows=")[-1].split("&")[0])
if MAX_NFT_NB/NFT_NUMBER_PER_PAGE == MAX_NFT_NB//NFT_NUMBER_PER_PAGE:
  scroll_down = np.arange((MAX_NFT_NB//NFT_NUMBER_PER_PAGE)-1)
else:
  scroll_down = np.arange(MAX_NFT_NB//NFT_NUMBER_PER_PAGE)
for i in tqdm(scroll_down, postfix=' Scrolling down on the main page'):
  while CURRENT_PAGE_NUMBER == (browser.current_url.split("page=")[-1]).split("&")[0]:
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  CURRENT_PAGE_NUMBER = (browser.current_url.split("page=")[-1]).split("&")[0]

100%|██████████| 1/1 [00:00<00:00,  1.18it/s,  Scrolling down on the main page]

CPU times: user 94.9 ms, sys: 11.8 ms, total: 107 ms
Wall time: 1.31 s


## 3. Récupération des liens des pages détaillées
- Détection des boutons donnant accès aux infos détaillées

In [5]:
%%time

detailed_pages = []
pbar = tqdm(total = MAX_NFT_NB, postfix=" Searching of NFT detailed pages")
BAR_LEVEL = 0
while len(detailed_pages) < MAX_NFT_NB:
  browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
  detailed_pages = browser.find_elements_by_xpath('//button[contains(text(),"BSC")]')
  pbar.update(max(0,len(detailed_pages)-BAR_LEVEL))
  BAR_LEVEL = len(detailed_pages)
pbar.close()

32it [00:08,  3.88it/s,  Searching of NFT detailed pages]                        

CPU times: user 104 ms, sys: 12.8 ms, total: 116 ms
Wall time: 8.32 s


## 4. Parcours des pages détaillés
- Récupération des données utiles sur les nft
- Nettoyage des chaînes de caractères
- Stockage des dictionnaires de données
- Fermeture du navigateur distant

In [6]:
%%time
rm_old_save=True
save_file = f'binance_{datetime.now()}.txt'.replace(":","")

for page in tqdm(detailed_pages, postfix=" Scraping of detailed NFT pages"):
  page.click()
  browser.switch_to.window(browser.window_handles[-1])
  add_to_json(parse_nft(browser), save_file)
  browser.close()
  browser.switch_to.window(browser.window_handles[-1])

if rm_old_save:
    [os.remove(file) for file in os.listdir('.') if file.find('.json')!=-1]    
os.rename(save_file, save_file.replace(".txt",".json"))
browser.quit()

100%|██████████| 32/32 [07:57<00:00, 14.94s/it,  Scraping of detailed NFT pages]


CPU times: user 8.8 s, sys: 2.04 s, total: 10.8 s
Wall time: 8min 5s


___

In [59]:
# Process the data
def process_data(nft):
    nft['id'] = str(hash(nft['contract_address']))
    nft['date'] = str(datetime.now())
    amount, currency = nft['currency'].split(' ')
    nft['amount'] = float(amount)
    nft['currency'] = currency
    return nft

In [60]:
# Necessary packages
import json
import requests

# Fetch the data
filename = save_file.replace("txt", "json")
with open(filename, 'r') as f:
    data = json.load(f)

# Apply processing
nfts = [*map(process_data, data)]

In [61]:
data[0]

{'id': '-4118167953241571505',
 'name': 'CAM 2D 1914 849/990',
 'image': 'https://public.nftstatic.com/static/nft/zipped/724d1bb0179d4df9aa7e40ddd79c8017_zipped.jpeg',
 'amount': 29.9,
 'currency': 'BUSD',
 'creator': 'CAM13',
 'date': '2021-09-22 14:05:18.924785',
 'description': 'Em 1914, foi realizado o primeiro torneio oficial de futebol de Minas Gerais: a Taça Bueno Brandão. O Atlético Mineiro disputou quatro partidas, ganhou três e empatou uma, tornando-se campeão invicto, o primeiro título de sua história. Esta camisa é parte da coleção de 13 camisas icônicas do Galo. Complete a coleção 2D e ganhe um NFT exclusivo, que não estará à venda, e um vídeo de agradecimento de um craque do Galo.',
 'contract_address': '0x1dDB2C0897daF18632662E71fdD2dbDC0eB3a9Ec',
 'link': 'https://www.binance.com/en/nft/goods/detail?productId=7570214&isProduct=1'}

In [64]:
success = 0
failure = 0
total = len(nfts)

for nft in nfts[:2]:
    url = 'http://api:8000/nft'
    headers = {'accept': 'application/json', 'Content-Type': 'application/json'}
    res = requests.post(url, data=json.dumps(nft), headers=headers)
    if res.status_code == 201:
        success += 1
    else:
        failure += 1
    print(f"Success={success}/{total}; Failure={failure}/{total} - {res.json()}")

Success=0/32; Failure=1/32 - {'detail': 'Method Not Allowed'}
Success=0/32; Failure=2/32 - {'detail': 'Method Not Allowed'}


In [65]:
res = requests.post(url, data=json.dumps(nft), headers=headers)

print(res.status_code)
print()
print(res.json())
print()
print(res.text)
print()
print(res.reason)

405

{'detail': 'Method Not Allowed'}

{"detail":"Method Not Allowed"}

Method Not Allowed


In [66]:
requests.get('http://api:8000/nfts').text

'[]'